In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

In [34]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224, 3))
base_model.trainable = False

In [35]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=x)

In [36]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [37]:
from PIL import Image
import os

dataset_path = r'C:\Users\Manvik\Downloads\Compressed\PetImages'

folders = ['Cat', 'Dog']

for folder in folders:
    folder_path = os.path.join(dataset_path, folder)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            img = Image.open(file_path)
            img.verify()  # check if image is OK
        except (IOError, SyntaxError):
            print(f"Removing corrupted file: {file_path}")
            os.remove(file_path)



In [38]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

train_generator = train_datagen.flow_from_directory(r'C:\Users\Manvik\Downloads\Compressed\PetImages',
                                                   target_size=(224,224),
                                                   batch_size=32,
                                                   class_mode='binary',
                                                   subset='training')

val_generator = train_datagen.flow_from_directory(r'C:\Users\Manvik\Downloads\Compressed\PetImages',
                                                   target_size=(224,224),
                                                   batch_size=32,
                                                   class_mode='binary',
                                                   subset='validation')

Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [39]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 571s 907ms/step - accuracy: 0.9665 - loss: 0.0865 - val_accuracy: 0.9758 - val_loss: 0.0622
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 562s 899ms/step - accuracy: 0.9743 - loss: 0.0623 - val_accuracy: 0.9764 - val_loss: 0.0686
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 647s 1s/step - accuracy: 0.9777 - loss: 0.0538 - val_accuracy: 0.9744 - val_loss: 0.0622
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 656s 1s/step - accuracy: 0.9788 - loss: 0.0528 - val_accuracy: 0.9774 - val_loss: 0.0597
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 922s 1s/step - accuracy: 0.9802 - loss: 0.0497 - val_accuracy: 0.9700 - val_loss: 0.0702


In [45]:
# Predict on a single image
from tensorflow.keras.preprocessing import image
import numpy as np

img_path = 'dog.jpeg'
img = image.load_img(img_path, target_size=(224,224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

prediction = model.predict(img_array)
if prediction[0][0] > 0.5:
    print("Dog")
else:
    print("Cat")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Dog
